In [ ]:
import numpy as np
import edf
from time import time
import sys
import matplotlib.pyplot as plt

data = np.load('./c10_data/train.npz')
t_imgs = np.float32(data['imgs'])/255.

# Reshape the train image data to (idx, h, w, channel)
t_imgs = t_imgs.reshape(50000, 32, 32, 3)
t_labels = np.float32(data['labels'])

data = np.load('./c10_data/test.npz')
v_imgs = np.float32(data['imgs'])/255.

# Reshape the valid image data to (idx, h, w, channel)
v_imgs = v_imgs.reshape(10000, 32, 32, 3)
v_labels = np.float32(data['labels'])

In [ ]:

########################################### Convolution layer#############################################
############################### Please implement the forward abd backward method in this class ############## 
class Conv:

    def __init__(self,f,k,stride=1,pad=0):
        edf.components.append(self)
        self.f = f
        self.k = k
        pad = np.array(pad)
        if pad.shape == ():
            self.xpad = self.ypad = pad
        else:
            self.ypad = pad[0]
            self.xpad = pad[1]
            
        self.stride=stride
        self.grad = None if f.grad is None and k.grad is None else edf.DT(0) 

    ####################### Please implement this function####################### 
    def forward(self):
        inshape = self.k.shape
        if self.value is None:
            owsize = np.ceil((inshape[1] - self.f.shape[0] + 1) / self.stride)
            ohsize = np.ceil((inshape[2] - self.f.shape[0] + 1) / self.stride)
            self.value = np.ndarray([inshape[0], owsize, ohsize, inshape[3]], 
                                    np.dtype(np.float64))
        
        # padding
        padded = np.ndarray([inshape[0], inshape[1] + 2 * self.pad, inshape[2] + 2 * self.pad, inshape[3]])
        padded.fill(0)
        padded[:, self.pad: inshape[1] + self.pad , self.pad : inshape[2] + self.pad ,:] = self.k.value
        yshape = self.value.shape
        ksize = self.f.shape[0]
        for bi in range(yshape[0]):
            for ci in range(yshape[3]):
                for wi in range(yshape[1]):
                    for hi in range(yshape[2]):
                        self.value[bi, wi, hi, ci] = np.multiply(padded[bi, 
                                                                        self.stride * wi : self.stride * wi + ksize,
                                                                        self.stride * hi : self.stride * hi + ksize,
                                                                        :], 
                                                                 self.f[:,:,:,ci])
                                                        .sum()
                
    ####################### Please implement this function#######################         
    def backward(self):
        if self.k.grad is None or self.f.grad is None:
            return
        fgrad = np.ndarray(self.f.value.shape, np.dtype(np.float64))
        fgrad.fill(0)
        kgrad = np.ndarray(self.k.value.shape, np.dtype(np.float64))
        kgrad.fill(0)
        
        kshape = self.k.value.shape
        fshape = self.f.value.shape
        yshape = self.value.shape
        for bi in range(kshape[0]):
            for ci in range(kshape[3]):
                for c2i in range(kshape[3]):
                    for wi in range(yshape[1]):
                        for hi in range(yshape[2]):
                            for wki in range(fshape[0]):
                                for hki in range(fshape[0]):
                                    xwi = self.stride * wi + wki
                                    xhi = self.stride * hi + whi
                                    kgrad[bi, xwi, xhi, ci] += self.grad[bi, wi, hi, c2i] * self.f.value[wki, hki, c2i, ci]
                                    fgrad[wki, hki, c2i, ci] += self.grad[bi, wi, hi, c2i] * self.k.value[bi, xwi, xhi, ci]
        self.f.grad += fgrad
        self.k.grad += kgrad

########################################### MaxPool layer#############################################
############################### Please implement the forward abd backward method in this class ##############             
class MaxPool:
    def __init__(self,x,ksz=2,stride=None):
        edf.components.append(self)
        self.x = x
        self.ksz=ksz
        if stride is None:
            self.stride=ksz
        else:
            self.stride=stride
        self.grad = None if x.grad is None else edf.DT(0)

    ####################### Please implement this function#######################     
    def forward(self):
        if self.value is None:
            xshape = self.x.value.shape
            self.value = np.ndarray([xshape[0], 
                                     xshape[1] / self.stride, 
                                     xshape[2] / self.stride, 
                                     xshape[3]], 
                                    self.x.value.dtype)
            self.xmaxs = {}
        sshape = self.value.shape
        for bi in range(sshape[0]):
            for ci in range(sshape[3]):
                for wi in range(sshape[1]):
                    for hi in range(sshape[2]):
                        xmax = np.NINF
                        for swi in range(self.stride):
                            for shi in range(self.stride):
                                # Record the maximal value and its location
                                xvalue = self.x.value[bi, wi * self.stride + swi, 
                                                      hi * self.stride + shi, ci]
                                record = (bi, wi * self.stride + swi, hi * self.stride + shi, ci)
                                if xvalue > xmax:
                                    xmax = xvalue
                                    self.xmaxs[(bi,wi,hi,ci)] = [record]
                                elif xvalue == xmax:
                                    self.xmaxs[(bi,wi,hi,ci)].append(record)
                        self.value[bi, wi, hi, ci] = xmax
                        
    ####################### Please implement this function#######################             
    def backward(self):
        if self.x.grad is None:
            return
        grad = np.ndarray(self.x.value.shape, np.dtype(np.float64))
        # for each grad, prop only to the max inputs
        sshape = self.value.shape
        for bi in range(sshape[0]):
            for ci in range(sshape[3]):
                for wi in range(sshape[1]):
                    for hi in range(sshape[2]):
                        gval = self.grad[bi,wi,hi,ci]
                        for xmaxp in self.xmaxs[(bi,wi,hi,ci)]:
                            grad[xmaxp[0],xmaxp[1],xmaxp[2],xmaxp[3]] = gval
        self.x.grad += grad
########################################### AvePool layer#############################################
############################### Please implement the forward abd backward method in this class ##############                             
class AvePool:
    def __init__(self,x,ksz=2,stride=None):
        edf.components.append(self)
        self.x = x
        self.ksz=ksz
        if stride is None:
            self.stride=ksz
        else:
            self.stride=stride
        self.grad = None if x.grad is None else edf.DT(0)
        
    ####################### Please implement this function#######################   
    def forward(self):
        if self.value is None:
            xshape = self.x.value.shape
            self.value = np.ndarray([xshape[0], 
                                     xshape[1]/self.stride,  
                                     xshape[2]/self.stride, 
                                     xshape[3]], 
                                    self.x.value.dtype)
        sshape = self.value.shape
        for bi in range(sshape[0]):
            for ci in range(sshape[3]):
                for wi in range(sshape[1]):
                    for hi in range(sshape[2]):
                        xsum = edf.DT(0)
                        for swi in range(self.stride):
                            for shi in range(self.stride):
                                xsum += self.x.value[bi, wi * self.stride + swi, 
                                                      hi * self.stride + shi, ci]
                        xavg = xsum / (np.square(self.stride))
                        self.value[bi, wi, hi, ci] = xavg
    ####################### Please implement this function#######################    
    def backward(self):
        if self.x.grad is None:
            return
        grad = np.ndarray(self.x.value.shape, np.dtype(np.float64))
        sshape = self.value.shape
        for bi in range(sshape[0]):
            for ci in range(sshape[3]):
                for wi in range(sshape[1]):
                    for hi in range(sshape[2]):
                        gval = self.grad[bi,wi,hi,ci]
                        for wsi in range(self.stride):
                            for hsi in range(self.stride):
                                grad[bi, wi * self.stride + wsi, hi * self.stride + hsi, ci] = gval
        
        self.x.grad += grad / np.square(self.stride)

In [ ]:
# for repeatability
np.random.seed(0)

# Inputs
inp = edf.Value()
lab = edf.Value()


prev_channel = 3 # RGB channel 
########################## Simple Convolution Nerual Network Model for Cifar 10 ##################################
##################################################################################################################
# please implement your main cnn model here, as described by the homework, you can mimic the previous code






# the standard classification layer, which you don't need to modify
pred = edf.SoftMax(pred)
loss = edf.Mean(edf.LogLoss(edf.Aref(pred,lab)))
acc = edf.Accuracy(pred,lab)


################################################################################################################## 
# evaluation bucket
bucket = 100
def eval_train():    
    
    # we only choose 1/5 of the train images for evaluation since evaluation the whole images is time consuming
    eval_imgs = t_imgs[::5]
    eval_labels = t_labels[::5]
    avg_acc = 0
    avg_loss = 0
    
    for seq in range(bucket):
        inp.set(eval_imgs[seq::bucket])
        lab.set(eval_labels[seq::bucket])
        edf.Forward()
        avg_acc += acc.value
        avg_loss += loss.value
    
    return avg_acc/bucket, avg_loss/bucket
        
def eval_test():
    
    avg_acc = 0
    avg_loss = 0
    for seq in range(bucket):
        inp.set(v_imgs[seq::bucket])
        lab.set(v_labels[seq::bucket])
        edf.Forward()
        avg_acc += acc.value
        avg_loss += loss.value
    
    return avg_acc/bucket, avg_loss/bucket

# initial accuracy 
random_acc, random_loss = eval_test()
print("Random test loss = %.4f, accuracy = %.4f" % (random_loss, random_acc))


################################################# train loop ######################################################
ep = 0
epoch = 10
batch = 100
train_loss = []; train_acc = []; test_loss =[]; test_acc = []
stime = time()
batches = range(0, len(t_labels), batch)

while ep < epoch:

    # randon shuffle the train data in each epoch
    perm = np.random.permutation(len(t_labels))

    for k in batches:
        inp.set(t_imgs[perm[k:k+batch]])
        lab.set(t_labels[perm[k:k+batch]])
        edf.Forward()
        edf.Backward(loss)
        edf.Adam()
        
    # evaluate on trainset
    t_acc, t_loss = eval_train()
    print("Epoch %d: train loss = %.4f [%.3f secs]" % (ep, t_loss,time()-stime))
    train_loss.append(t_loss)
    train_acc.append(t_acc)

    # evaluate on testset
    v_acc, v_loss = eval_test()
    print("test accuracy = %.4f" % v_acc)
    test_loss.append(v_loss)
    test_acc.append(v_acc)
    stime = time()
    ep += 1      


# plot
plt.figure(1)
plt.xlabel("epochs")
plt.ylabel("loss")
plt.plot(np.arange(len(test_loss)), test_loss, color='red')
plt.plot(np.arange(len(train_loss)), train_loss, color='blue')
plt.legend(['test loss', 'train loss'], loc='upper right')
plt.show()

plt.figure(2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.plot(np.arange(len(test_acc)), test_acc, color='red')
plt.plot(np.arange(len(train_acc)), train_acc, color='blue')
plt.legend(['test acc', 'train acc'], loc='lower right')
plt.show()